<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 2: Sistemas de ecuaciones numéricas lineales

In [1]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [2]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

In [3]:
from anmi.genericas import norma, print_verbose, norma_1, norma_inf, norma_2

from numpy.linalg import cond as numero_condicion
from anmi.T2 import descomposicion_LU, descomposicion_LDU, cholesky, gram_schmidt, householder, factorizacion_QR
from anmi.T2 import metodo_iterativo, criterio_m_matriz, criterio_SOR, criterio_simetrica_definida_positiva, criterio_diagonal_dominante, criterio_radio_espectral

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [4]:
help(numero_condicion)

Help on function cond in module numpy.linalg:

cond(x, p=None)
    Compute the condition number of a matrix.
    
    This function is capable of returning the condition number using
    one of seven different norms, depending on the value of `p` (see
    Parameters below).
    
    Parameters
    ----------
    x : (..., M, N) array_like
        The matrix whose condition number is sought.
    p : {None, 1, -1, 2, -2, inf, -inf, 'fro'}, optional
        Order of the norm:
    
        =====  ============================
        p      norm for matrices
        =====  ============================
        None   2-norm, computed directly using the ``SVD``
        'fro'  Frobenius norm
        inf    max(sum(abs(x), axis=1))
        -inf   min(sum(abs(x), axis=1))
        1      max(sum(abs(x), axis=0))
        -1     min(sum(abs(x), axis=0))
        2      2-norm (largest sing. value)
        -2     smallest singular value
        =====  ============================
    
        inf mea

In [5]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

Matrix([
[1, 2, 3],
[2, 3, 1],
[3, 2, 4]])

In [6]:
numero_condicion(np.array(M).astype(int))

6.960250455451664

In [7]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

array([[-0.50127996, -0.23313726,  0.83328592],
       [-0.45615124,  0.88953592, -0.02553206],
       [-0.73528528, -0.39290311, -0.55225239]])

In [8]:
numero_condicion(M_ort)

1.0000000000000004

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [9]:
help(descomposicion_LU)

Help on function descomposicion_LU in module anmi.T2.funcs_LUD:

descomposicion_LU(m, rhs=None, verbose=True, permutar_max=False)
    Esta función realiza el algoritmo de triangulación de Gauss.
        Para ello vamos a ir aplicando paso a paso el algoritmo tal
        cual se hace manualmente,
        y aplicamos los cambios de columnas necesarios si hay
        que aplicar permutaciones.
        Por defecto, si encontramos un 0 en la diagonal aplicamos
        permutar_max para esa
        fila, y devolvemos la matriz de permutaciones.
    
    Args:
        m (matriz): Matriz a descomponer
        rhs (matriz, optional): Lado derecho de la ecuación. Default es [1, 1, ..., 1]
        verbose (bool, optional): Aplicar verbose.
        permutar_max (bool, optional): Aplicar permutación por filas aun si no es necesario. Defaults to False.
    
    Returns:
        dict: {"P": matriz de permutaciones, "L": matriz L, U": matriz U, "r": matriz rhs adecuada a U}



In [10]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [11]:
descomposicion_LU(M, permutar_max=False)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 0]
 [3 2 1 0]
 [2 4 1 0]]

Fila 0
A 0, 0

Fila 1
A 1, 0
||||||||||||||||||||||||
 Columna 0
a_1,0 = 3
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [2 4 1]]
r = 
[[0]
 [0]
 [0]]

Fila 2
A 2, 0
||||||||||||||||||||||||
 Columna 0
a_2,0 = 2
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 -4 -7]]
r = 
[[0]
 [0]
 [0]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]
r = 
[[0]
 [0]
 [0]]
================= FORMA FINAL
a_3,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]

L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]

U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]

r = 
[[0]
 [0]
 [0]]



{'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1,   4,     4],
 [0, -10,   -11],
 [0,   0, -13/5]]), 'r': Matrix([
 [0],
 [0],
 [0]])}

In [12]:
descomposicion_LDU(M, permutar_max=False)

{'D': Matrix([
 [1,   0,     0],
 [0, -10,     0],
 [0,   0, -13/5]]), 'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1, 4,     4],
 [0, 1, 11/10],
 [0, 0,     1]])}

In [13]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 1]
 [3 2 1 1]
 [2 4 1 1]]

Fila 0
A 0, 0
El índice de permutación es igual a la fila a permutar.

Fila 1
A 1, 0
Permutamos fila 1 con 2
U antes:
 [[1 4 4]
 [3 2 1]
 [2 4 1]]
P antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
U despues:
 [[1 4 4]
 [2 4 1]
 [3 2 1]]
P despues:
 [[1 0 0]
 [0 0 1]
 [0 1 0]]
L antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
L despues:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Fila 1
A 1, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_1,0 = 2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [3 2 1]]
r = 
[[1]
 [-1]
 [1]]

Fila 2
A 2, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_2,0 = 3
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [3 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [0 -10 -11]]
r = 
[[1]
 [-1]
 [-2]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 5/2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[

{'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1,  4,    4],
 [0, -4,   -7],
 [0,  0, 13/2]]), 'r': Matrix([
 [  1],
 [ -1],
 [1/2]])}

In [14]:
descomposicion_LDU(M, permutar_max=True)

{'D': Matrix([
 [1,  0,    0],
 [0, -4,    0],
 [0,  0, 13/2]]), 'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1, 4,   4],
 [0, 1, 7/4],
 [0, 0,   1]])}

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [15]:
help(cholesky)

Help on function cholesky in module anmi.T2.funcs_LUD:

cholesky(m, verbose=False)
    Primero comprobamos que los menores sean positivos. Eso es equivalente que
    sus autovalores sean positivos. Por simplificar, si el determinante es negativo
    ya descartamos que sea factorizable, y saltamos el warning.



In [16]:
M = mat(((2, -1, 0), (-1, 2, -2), (0, 2, 1)))
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [17]:
cholesky(M)

AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.
AVISO!!! La matriz no es Cholesky-zable. 
 L = 
 [[sqrt(2) 0 0]
 [-sqrt(2)/2 sqrt(6)/2 0]
 [0 2*sqrt(6)/3 sqrt(15)*I/3]]

 L*L.T = 
 {np.array(m_chol * m_chol.T)}


Matrix([
[   sqrt(2),           0,            0],
[-sqrt(2)/2,   sqrt(6)/2,            0],
[         0, 2*sqrt(6)/3, sqrt(15)*I/3]])

In [18]:
cholesky(M + M.T)

Matrix([
[ 2,       0,       0],
[-1, sqrt(3),       0],
[ 0,       0, sqrt(2)]])

In [19]:
M + M.T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [20]:
cholesky(M + M.T) * cholesky(M + M.T).T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [21]:
# Podemos hacer también Cholesky a una matriz con símbolos!
x = symbols('x')

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

Matrix([
[1, x, 1],
[x, 2, 1],
[1, 1, 3]])

In [22]:
cholesky(Mx)

Matrix([
[1,                      0,                                    0],
[x,         sqrt(2 - x**2),                                    0],
[1, (1 - x)/sqrt(2 - x**2), sqrt((-3*x**2 + 2*x + 3)/(2 - x**2))]])

### Ortogonalización de Gram-Schmidt




In [23]:
help(gram_schmidt)

Help on function gram_schmidt in module anmi.T2.funcs_gram_schmidt:

gram_schmidt(m, verbose=False)
    La ortogonalización produce una matriz ortogonalizada por columnas.
    p es la matriz ortogonal y p_norm es la ortonormal
    c es la matriz triangular tal que cij = aj·pi/||pi||^2 (los coeficientes de ortogonalización).
    Estos coeficientes se usan para la factorización QR.



In [24]:
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [25]:
GS = gram_schmidt(M)
GS['P']

Matrix([
[ 2, 3/5, -22/29],
[-1, 6/5, -44/29],
[ 0,   2,  33/29]])

In [26]:
GS['Pn']

Matrix([
[ 2/5,  3/29, -2/11],
[-1/5,  6/29, -4/11],
[   0, 10/29,  3/11]])

In [27]:
GS['c']

Matrix([
[0, -4/5,   2/5],
[0,    0, -2/29],
[0,    0,     0]])

In [28]:
GS['P'][:, 1].T * GS['P'][:, 2]

Matrix([[0]])

In [29]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [30]:
GSx = gram_schmidt(Mx)
GSx['P']

Matrix([
[1, 1 - x/2, 0],
[1, x/2 - 1, 0],
[0,       0, 3]])

### Transformación de Householder

La transformación de Householder es una transformación para pasar de un vertor $x$ a un vector $y$. Para ello se toma el vector $e$, que sería el eje de transformación, y la matriz aplicación es $H = I - 2ee^t$.

En este caso, $$e = \pm \frac{x - y}{\vert\vert x - y \vert\vert}$$

Para la transformación de Householder $x$ e $y$ **tienen que tener la misma norma**. El vector resultante $e$ tiene norma 1.


In [31]:
help(householder)

Help on function householder in module anmi.T2.funcs_householder_QR:

householder(x, y, signo='+', normalizar=False)
    Aplica la transformación de Householder. x e y pueden ser vectores fila o columna,
    en cuyo caso aplicamos la transformación de una forma u otra.
    
    Args:
        x (vector): vector origen
        y (vector): vector destino
        signo (str, optional): + o -, si - aplica la transformación inversa. Defaults to "+".
        normalizar (bool, optional): Normaliza los vectores para la transformación. Defaults to False.
    
    Returns:
        dict: {'H': la aplicación de transformación, 'e': el vector de eje de cambio.}



In [32]:
v0 = mat(((1, 0)))
vf = mat((1, 1))
vf = vf/norma(vf)

In [33]:
H, e = householder(v0, vf, normalizar=False)
H

Matrix([
[sqrt(2)/2,  sqrt(2)/2],
[sqrt(2)/2, -sqrt(2)/2]])

In [34]:
e

Matrix([
[           sqrt(2 - sqrt(2))/2],
[-sqrt(2)/(2*sqrt(2 - sqrt(2)))]])

In [35]:
# Ejercicio 10
t = symbols('t')

v0 = mat(((1, 0)))
vf = mat((cos(t), sin(t)))

H, e = householder(v0, vf, normalizar=False)

In [36]:
H

Matrix([
[cos(t),  sin(t)],
[sin(t), -cos(t)]])

In [37]:
e

Matrix([
[      sqrt(2 - 2*cos(t))/2],
[-sin(t)/sqrt(2 - 2*cos(t))]])

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

In [38]:
help(factorizacion_QR)

Help on function factorizacion_QR in module anmi.T2.funcs_householder_QR:

factorizacion_QR(m, verbose=True, metodo='gram_schmidt')
    Aplica el método QR para la factorización de la matriz m. m = QR, con Q ortogonal y R triangular superior.
    
    Args:
        m (matriz): matriz a factorizar.
        verbose (bool, optional): Imprime mensajes informativos. Defaults to True.
        metodo (str, optional): "gram_schmidt" o "householder".
        Householder funciona mejor para vectores proximos,
        donde gram-schmidt deja de ser estable. Defaults to "gram_schmidt".
    
    Returns:
        dict: {'Q', 'R' , 'D'} ('D' en gram-schmidt solo).



In [39]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [40]:
factorizacion_QR(M, metodo='householder')

Aplicamos QR con Householder
Q es 
[[1 0 0]
 [0 0 -1]
 [0 1 0]]
R es 
[[2 -1 0]
 [0 2 -1]
 [0 0 2]]


{'Q': Matrix([
 [1, 0,  0],
 [0, 0, -1],
 [0, 1,  0]]), 'R': Matrix([
 [2, -1,  0],
 [0,  2, -1],
 [0,  0,  2]])}

In [41]:
A1 = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [42]:
A1 = mat(((1, 2, 3), (4, 5, 6), (7, 8, 9)))
A1

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

In [43]:
dqr = factorizacion_QR(A1, metodo='householder')

Aplicamos QR con Householder
AVISO!!! Hx != y. Para Householder las normas tienen que ser iguales.
Q es 
[[sqrt(66)/66
  3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [2*sqrt(66)/33
  (-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-65*sqrt(6) + 7*sqrt(66) + 72)/(3*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [7*sqrt(66)/66
  (-12*sqrt(66) - 77 + 65*sqrt(11))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]]
R es 
[[sqrt(66) 13*sqrt(66)/11 15*sqrt(66)/11]
 [0
  3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))
  6*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))]
 [0 0 0]]


In [44]:
dqr['Q']

Matrix([
[  sqrt(66)/66, 3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[2*sqrt(66)/33,   (-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-65*sqrt(6) + 7*sqrt(66) + 72)/(3*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[7*sqrt(66)/66,   (-12*sqrt(66) - 77 + 65*sqrt(11))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), (-72 - 7*sqrt(66) + 65*sqrt(6))/(6*(-65 + 7*sqrt(11) + 12*sqrt(6)))]])

In [45]:
dqr['R']

Matrix([
[sqrt(66),                                                           13*sqrt(66)/11,                                                           15*sqrt(66)/11],
[       0, 3*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6))), 6*(-65*sqrt(11) + 77 + 12*sqrt(66))/(11*(-65 + 7*sqrt(11) + 12*sqrt(6)))],
[       0,                                                                        0,                                                                        0]])

In [46]:
simplify(dqr['Q'].T * dqr['Q'])

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [47]:
simplify(dqr['Q'] * dqr['R'])

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

### Métodos iterativos



Los métodos iterativos son métodos para resolver el sistema $Ax=b$. Todos los métodos se basan en el mismo criterio, que es emplear una matriz $H$ tal que, para una iteración:
$$x^{(k+1)} = Hx^{(k)} + b'$$

Si tomamos la matriz $A = D - L - U$ (donde $D$ es la diagonal de $A$, y $L$ y $U$ son matrices triangulares de elementos de $A$), la ecuación anterior queda:
$$Mx^{(k+1)} = (M-A)x^{(k)} + b$$

Y simplemente hay que despejar para $x^{(k+1)}$ (pues $x^{(k)}$ lo conocemos) y repetir el procedimiento hasta convergencia:
$$x^{(k+1)} = M^{-1}(M-A)x^{(k)} + M^{-1}b$$

Según el método, la matriz difiere: para Jacobi es $M = D$, para Gauss-Seidel es $M = D-L$ y para sobre-relajación (ros) es $M = \frac{1}{\omega}D - L$.

In [48]:
help(metodo_iterativo)

Help on function metodo_iterativo in module anmi.T2.funcs_met_iters:

metodo_iterativo(A, b=None, x0=None, metodo='jacobi', w=1.5, n_iter=10, verbose=True)
    Aplica el método iterativo designado
    
    Args:
        A (matriz): Matriz de valores
        b (vector, optional): Vector de rhs. Por defecto es 1, 1, ..., 1.
        x0 (vector, optional): Vector con elementos de la primera iteración. Por defecto es 1, 1, ..., 1.
        metodo (str, optional): método de resolución, puede ser "jacobi", "gs" o "sor".
        w (float, optional): Peso para método sor. Defaults to 1.5.
        n_iter (int, optional): Número de iteraciones del método. Defaults to 10.
        verbose (bool, optional): Imprime resultados intermedios. Defaults to True.
    
    Returns:
        dict: 'x': vector de resultados para Ax=b, 'diff': diferencia entre Ax y b para cada iteración.



In [49]:
A = mat([[2, 1, 1], [1, -2, 1], [1, 1, 2]])
b = mat([[1, 1, 1]]).T

In [50]:
diter = metodo_iterativo(A, b, n_iter=200, verbose=True)
np.array(diter['x'], dtype=float)

||Criterio de radio espectral||
Los autovalores son [1/2, -1/4 - sqrt(7)*I/4, -1/4 + sqrt(7)*I/4]. Si el mayor autovalor es < 1, entonces el método converge.
||Criterio de Diagonal Dominante||
 Si la matriz es dominante por filas, los métodos de Jacobi y Gauss-Seidel convergen.
La fila 0 NO es dominante por filas: diff = 0.
||Criterio de Sim Def Pos||
 Si la matriz es simétrica y definida positiva, el método de Gauss-Seidel es convergente.
El determinante de A es -8.
La matriz NO es DEFINIDA POSITIVA (el determinante no es positivo).
||Criterio SOR||
 Si la matriz es simétrica y definida positiva y w in (0, 2) el método SOR es convergente.
Si w no (0, 2) el método SOR no converge.
||Criterio M matriz||
 Si la A es M-matriz entonces las descomposiciones de Jacobi y Gauss-Seidel son convergentes.
A^-1 >= 0
aij < 0 para todo i =/= j
A^-1 < 0. La matriz NO CUMPLE el criterio
La matriz tiene elementos no diagonales positivos. NO CUMPLE el criterio.


array([[ 0.375],
       [-0.125],
       [ 0.375]])

In [51]:
# EJERCICIO 12
a = symbols('a')

A = mat([[4, 1, a], [1, 4, 1], [a, 1, 4]])
metodo_iterativo(A, metodo='gs', n_iter=3)

||Criterio de radio espectral||
Los autovalores son [a**2/32 - a/128 - sqrt((4*a - 1)*(4*a**3 - a**2 + 16*a - 64))/128 + 1/16, a**2/32 - a/128 + sqrt((4*a - 1)*(4*a**3 - a**2 + 16*a - 64))/128 + 1/16, 0]. Si el mayor autovalor es < 1, entonces el método converge.
||Criterio de Diagonal Dominante||
 Si la matriz es dominante por filas, los métodos de Jacobi y Gauss-Seidel convergen.
La matriz tiene complejos o simbolos. Hay que verificar el criterio a mano.
||Criterio de Sim Def Pos||
 Si la matriz es simétrica y definida positiva, el método de Gauss-Seidel es convergente.
El determinante de A es -4*a**2 + 2*a + 56.
No podemos determinar la positividad porque hay símbolos o complejos.
||Criterio SOR||
 Si la matriz es simétrica y definida positiva y w in (0, 2) el método SOR es convergente.
Si w no (0, 2) el método SOR no converge.
||Criterio M matriz||
 Si la A es M-matriz entonces las descomposiciones de Jacobi y Gauss-Seidel son convergentes.
A^-1 >= 0
aij < 0 para todo i =/= j
La ma

{'diff': [Abs(-a*(1/64 - a/16) - 3/4) + Abs(-a**2/4 - 4*a*(1/64 - a/16) + a/16) + Abs(a*(-a*(1/64 - a/16) + 1/4) - 15*a/16 - 1),
  Abs(15*a*(1/64 - a/16)/16 + (1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1) - 3/64) + Abs(a*(1/64 - a/16)/4 - 15*a*(-a*(1/64 - a/16) + 1/4)/16 + a*(-a*(1/64 - a/16)/16 + (1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1) + 13/64) - 15*a/256 + 1/8) + Abs(a*(-a*(1/64 - a/16) + 1/4)/16 + a*(-a*(-a*(1/64 - a/16) + 1/4)/4 - a/64 + 1/4) + a/256 + 4*(1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1) - 1/16),
  Abs(15*a*(1/64 - a/16)/256 - 15*(1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1)/16 + (1/64 - a/16)*(-a*(1/64 - a/16)/4 - a*(-a*(1/64 - a/16) + 1/4)/16 - a*(-a*(1/64 - a/16)/16 + (1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1) + 13/64) - a/256 + 7/8) - 3/1024) + Abs(-7*a*(1/64 - a/16)/32 - 15*a*(-a*(1/64 - a/16) + 1/4)/256 - 15*a*(-a*(1/64 - a/16)/16 + (1/64 - a/16)*(-a*(-a*(1/64 - a/16) + 1/4) - a/16 + 1) + 13/64)/16 + a*(-a*(1/6

In [52]:
# EJERCICIO 14
A = mat([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
norma_1(A)
norma_inf(A)
d = [simplify(i) for i in list((A.T * A).eigenvals().keys())]
# norma_2(A)

In [53]:
A

Matrix([
[-11/27, 20/27,  8/27],
[ 20/27, 16/27, -8/27],
[  8/27, -8/27,  5/27]])

In [54]:
A.T * A

Matrix([
[   65/81,   4/81, -208/729],
[    4/81,  80/81,   -8/729],
[-208/729, -8/729,    17/81]])

In [55]:
simplify((A.T * A - a * eye(3)))

Matrix([
[65/81 - a,      4/81,  -208/729],
[     4/81, 80/81 - a,    -8/729],
[ -208/729,    -8/729, 17/81 - a]])

# Ejercicios

#### Ejercicio 14
Calcular $\vert\vert A\vert\vert_1, \vert\vert A\vert\vert_2, \vert\vert A\vert\vert_\infty$ de la matriz

In [62]:
A = Matrix([[-11, 20, 8], [20, 16, -8], [8, -8, 5]]) / 27
A

Matrix([
[-11/27, 20/27,  8/27],
[ 20/27, 16/27, -8/27],
[  8/27, -8/27,  5/27]])

In [72]:
np.sum(abs(A), 0), norma_1(A)

(array([13/9, 44/27, 7/9], dtype=object), 44/27)

In [73]:
np.sum(abs(A), 1), norma_inf(A)

(array([13/9, 44/27, 7/9], dtype=object), 44/27)

In [84]:
# Para la norma_2 podemos usar norma_2(), pero vamos a hacerlo a mano. Para ello:
# 1) Hallamos A.T * A
AA = simplify(A.T * A)
AA

Matrix([
[   65/81,   4/81, -208/729],
[    4/81,  80/81,   -8/729],
[-208/729, -8/729,    17/81]])

In [86]:
# En este punto los resultados difieren, así que usaremos sus resultados intermedios
AA = Matrix([[65, 4, -32], [4, 80, 8], [-32, 8, 17]]) / 81

In [90]:
# 2) Hallamos los autovalores
factor(det(AA - symbols('lambda') * eye(3)))

-lambda*(lambda - 1)**2

In [97]:
# Los autovalores son 0 y 1. Luego nos quedamos con 1
rho = max(solve(det(AA - symbols('lambda') * eye(3))))
norma_2 = sqrt(rho)
norma_2

1

#### Ejercicio 15
La matriz de Hilbert $H_n$ tiene como coeficientes $h_{ij} = \frac{1}{i + j -1}$
Determinar el límite para $\vert\vert \; \vert\vert_1$ y $\vert\vert \; \vert\vert_\infty$: $\lim_{n\to \infty}\vert\vert H_n \vert\vert$.

In [103]:
def Hn(n):
    hn = zeros(n, n)
    for row in range(n):
        for col in range(n):
            hn[row, col] = S(1) / S(row + col + 1) # porque i y j les sumamos 1

    return hn

In [105]:
# Vamos a crear unas matrices de prueba

In [106]:
Hn(7)

Matrix([
[  1, 1/2, 1/3,  1/4,  1/5,  1/6,  1/7],
[1/2, 1/3, 1/4,  1/5,  1/6,  1/7,  1/8],
[1/3, 1/4, 1/5,  1/6,  1/7,  1/8,  1/9],
[1/4, 1/5, 1/6,  1/7,  1/8,  1/9, 1/10],
[1/5, 1/6, 1/7,  1/8,  1/9, 1/10, 1/11],
[1/6, 1/7, 1/8,  1/9, 1/10, 1/11, 1/12],
[1/7, 1/8, 1/9, 1/10, 1/11, 1/12, 1/13]])

Vemos que la matriz es simétrica, luego norma 1 y norma inf serán iguales.
La columna / fila con mayor suma es la primera, pues:
$A[1:, 0] = A[:-1, 1]$ y $A[0, 0] > A[-1, 1]$

Y por inducción en el resto de columnas se ve que $\sum A[:, i] > \sum A[:, i+1]$. 

El valor para $n$ de la primera fila es $\sum A[:, 0] = \sum_i^n 1/i$, que es la serie harmónica.

Para $n \to \infty$, sabemos que $\sum_i^n 1/i \to \infty$, luego $\lim_{n\to \infty}\vert\vert H_n \vert\vert = \infty$

#### Ejercicio 16
Se considera la matriz 

$$ A = \begin{pmatrix}a & 1+a\\0 & a\end{pmatrix}$$

con $a > 0 \in \mathbb{R}$.

1) Calcular el número de condición de $A$ en la norma $\vert \vert A \vert \vert_\infty$$

In [109]:
# El núemro de condición es ||A||·||A^-1||
a = symbols('a')

A = Matrix([[a, 1+a], [0, a]])
A

Matrix([
[a, a + 1],
[0,     a]])

In [113]:
Ainv = A.inv()
Ainv

Matrix([
[1/a, -1/a - 1/a**2],
[  0,           1/a]])

In [119]:
# Norma_inf A es a + a + 1 = 1 + 2a
ninfA = 1 + 2*a

# Norma_inf Ainv es 1/a + 1/a + 1/a2 = 
ninfAinv = 1/a + 1/a + 1/a**2

ninfA, ninfAinv

(2*a + 1, 2/a + a**(-2))

In [122]:
condicion = ninfA * ninfAinv
simplify(condicion)

(2*a + 1)**2/a**2

In [127]:
simplify(1 - factor(condicion))

1 - (2*a + 1)**2/a**2

2) Estimar el error relativo de la solución del sistema lineal perturbado
$$(A + \delta A)(x + \delta x) = b + \delta b$$

Por el lema de Banach tenemos que 

$$\frac{\vert\vert \delta x\vert\vert}{\vert\vert x\vert\vert} = \frac{cond(A)}{1-cond(A)\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}}\left(
    \frac{\vert\vert \delta b\vert\vert}{\vert\vert b\vert\vert} + 
    \frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}\right)$$


Para este ejercicio es 
$$\frac{\vert\vert \delta x\vert\vert}{\vert\vert x\vert\vert} = \frac{
    (2a+1)^2
}{(a^2 - (2a+1)^2)\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}}\left(
    \frac{\vert\vert \delta b\vert\vert}{\vert\vert b\vert\vert} + 
    \frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert}\right)$$

De lo cual se cumple que 
$$\frac{\vert\vert \delta A\vert\vert}{\vert\vert A\vert\vert} < \frac{1}{cond(A)}$$

#### Ejercicio 17
Calcular la factorización LU de la matriz $A$ con pivote parcial con $0 < \alpha < \frac{\pi}{4}$.

In [130]:
a = symbols('alpha')
A = Matrix([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, cos(a), -sin(a)], [0, 0, sin(a), cos(a)]])
A

Matrix([
[0, 1,          0,           0],
[1, 0,          0,           0],
[0, 0, cos(alpha), -sin(alpha)],
[0, 0, sin(alpha),  cos(alpha)]])

En este caso tenemos que la función LU nos falla porque el máximo de la matriz "no existe". Vamos a programarlo.
